In [1]:
# -*- encoding:urf-8 -*-
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table like and matrices
import pandas as pd
import numpy as np

# Modeling Helper
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno

# Configure visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 10,8

poi_path = '../../UAI_data/input/poi_re.csv'
train_Aug_path = '../../UAI_data/input/train_Aug_re.csv'
train_Jul_path = '../../UAI_data/input/train_July_re.csv'
weather_path = '../../UAI_data/input/weather.csv'
test_path = '../../UAI_data/input/test.csv'

poi = pd.read_csv(poi_path,encoding='gbk')
train_aug = pd.read_csv(train_Aug_path,encoding='gbk')
train_jul = pd.read_csv(train_Jul_path,encoding='gbk')
weather = pd.read_csv(weather_path)
test = pd.read_csv(test_path,encoding='gbk')

In [2]:
test.sort_values(['create_date','create_hour'])

,test_id,create_date,create_hour,start_geo_id_new,end_geo_id_new
277,277,2017-08-01,1,5,22
308,308,2017-08-01,1,24,251
311,311,2017-08-01,1,24,29
436,436,2017-08-01,1,47,110
443,443,2017-08-01,1,16,27
450,450,2017-08-01,1,24,63
452,452,2017-08-01,1,57,85
506,506,2017-08-01,1,69,60
515,515,2017-08-01,1,99,3
531,531,2017-08-01,1,88,58


In [3]:
train_jul

,create_date,create_hour,status,estimate_money,estimate_distance,estimate_term,driver_id,member_id,start_geo_id,end_geo_id,id
0,2017-07-01,0,2,140.00,20099.0,18.0,0,0,5,0,0
1,2017-07-01,0,2,78.00,9000.0,18.0,1,1,22,1,1
2,2017-07-01,0,1,86.23,10323.0,20.0,2,2,39,2,2
3,2017-07-01,0,1,81.88,14197.0,27.0,3,3,55,3,3
4,2017-07-01,2,1,217.01,39150.0,40.0,2,4,195,4,4
5,2017-07-01,2,1,55.69,8360.0,16.0,4,5,22,1,5
6,2017-07-01,6,2,188.00,40184.0,47.0,5,6,206,5,6
7,2017-07-01,8,2,56.00,10953.0,21.0,6,7,55,6,7
8,2017-07-01,9,2,25.00,2889.0,5.0,7,8,68,7,8
9,2017-07-01,9,2,31.00,4415.0,8.0,8,9,63,8,9


In [4]:
coord = train_jul.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['start_geo_id'].agg({'count':'count'})
coord['count'].value_counts()

1      341888
2       82820
3       33273
4       16108
5        9035
6        5417
7        3410
8        2316
9        1644
10       1124
11        812
12        625
13        466
14        401
15        282
16        261
17        193
18        186
19        144
20        107
21         96
22         96
23         63
24         56
27         56
25         46
26         36
29         33
28         30
30         29
        ...  
59          3
55          3
72          3
51          3
53          3
63          2
65          2
64          2
60          2
52          2
144         1
165         1
131         1
125         1
86          1
123         1
101         1
96          1
89          1
178         1
76          1
75          1
74          1
73          1
71          1
69          1
68          1
62          1
61          1
198         1
Name: count, dtype: int64